In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = "customerID STRING, gender STRING, SeniorCitizen DOUBLE, Partner STRING, Dependents STRING, tenure INT, PhoneService STRING, MultipleLines STRING, InternetService STRING, OnlineSecurity STRING, OnlineBackup STRING, DeviceProtection STRING, TechSupport STRING, StreamingTV STRING, StreamingMovies STRING, Contract STRING, PaperlessBilling STRING, PaymentMethod STRING, MonthlyCharges DOUBLE, TotalCharges DOUBLE, Churn STRING"

data = spark.read.options(header=True).schema(schema).csv(
    "/Volumes/workspace/default/customer_churn/customer_churn.csv"
)
display(data)